# Decision Trees Exercise
In this exercise you will show that ID3 is sub-optimal. Implement a simple version of Decision Tree, and will then apply a Decision Tree classsifier on the MNIST hand written digits dataset that we already saw.


## 1. Suboptimality of ID3
Consider the following training set, where $\mathcal{X} = \{0, 1\}^3$ and $\mathcal{Y} =\{0, 1\}$:

$$
\begin{aligned}
((1, 1, 1), 1)\\
((1, 0, 0), 1)\\
((1, 1, 0), 0)\\
((0, 0, 1), 0)
\end{aligned}
$$

Suppose we wish to use this training set in order to build a decision tree of depth 2 (i.e. for each
input we are allowed to ask two questions of the form "$x_i = 0$?" before deciding on the label).

1. Suppose we run the ID3 algorithm up to depth 2 (namely, we pick the root node and its
children according to the algorithm, but instead of keeping on with the recursion, we stop
and pick leaves according to the majority label in each subtree, once we reach depth 2). 
Assume that the subroutine used to measure the quality of each feature is based on the information gain, and that if two features get the same score, one of them is picked arbitrarily. 
Show that the training error of the resulting decision tree is at least 1/4.
2. Find a decision tree of depth 2, which attains zero training error.


#### Answer

##### Question 1:

Let's calculate Gain for each feature:
1. For 1st feature: 0.39
2. For 2nd feature: 0
3. For 3rd feature: 0


So, as a root we choose 1t feature and now we have next nodes:

Feature 1 == 1, node A:
$$
\begin{aligned}
((1, 1, 1), 1)\\
((1, 0, 0), 1)\\
((1, 1, 0), 0)\\
\end{aligned}
$$

Feature 1 == 0, node B:
$$
\begin{aligned}
((0, 0, 1), 0)
\end{aligned}
$$

Node B is a final leaf with 0 entropy. 

For the node A, let's calculate gain for next 2 features:
1. For 2nd feature: 0.5
2. For 3nd feature: 0.5

So, we could should one of them with the same gain.

Let's now calculate the training error:

In the final tree from this algorythm we gor 3 leafes: {0, 1, [1,0]} or {0,[1,0],0} in both cases 1 of 4 points is assigned incorrected, so the error is 1/4




#### Answer

##### Question 2:

1. Split by 2nd feature in subsets

A:
$$
\begin{aligned}
((1, 1, 1), 1)\\
((1, 1, 0), 0)\\
\end{aligned}
$$

B:
$$
\begin{aligned}
((1, 0, 0), 1)\\
((0, 0, 1), 0)
\end{aligned}
$$

2. Then split A by the 3rd feature and B by the 1st feature and we got the set {1,0,1,0} with the train error is 0.


## 2. Implementing Decision Tree From Scratch
In this exercise you will need to implement a simple version of Decision Tree from scratch. Your decision tree will handle **continuous input and output** (this should actually work also for binary input attributes).

* Compelete the skeleton class below
  - `X` is a matrix of data values (rows are samples, columns are attributes)
  - `y` is a vector of corresponding target values
  - `min_leaf` is the minimal number of samples in each leaf node
  
* For splitting criterion, use either **"Train Squared Error Minimization (Reduction in Variance)"** or **"Train Absolute Error Minimization"** (choose one). Whatever you choose, make sure you implement the splitting point decision efficiently (in $O(nlgn)$ time).

* The `predict` function will use mean of the target values in the leaf node matching each row of the given `X`. The result is a vector of predictions matching the number of rows in `X`.

* To check your decision tree implementation, use the boston dataset (`from sklearn.datasets import load_boston`) split the data set into train and test using (`from sklearn.model_selection import train_test_split`)

  - Use the following to estimate what are the best hyper parameters to use for your model
```
    for min_leaf in [1,5,10,100]:
      dt = DecisionTree(X, y, n, sz, min_leaf)
      mse = # mean square error over test set
      print("min_leaf:{0} --- oob mse: {1}".format(min_leaf, mse))
```
  
  - Using your chosen hyperparameters as a final model, plot the predictions vs. true values of all the samples in the training set . Use something like:
  ```
  y_hat = dt.predict(X_train)  # forest is the chosen model
  plt.scatter(y_hat, y_test)
  ```

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.base import BaseEstimator, ClassifierMixin

In [113]:
class DecisionTree(BaseEstimator,ClassifierMixin):
  def __init__(self, min_leaf):
    self.X = None
    self.y = None
    self.min_leaf = min_leaf
    self.right = None
    self.left = None
    self.feature = None
    self.feature_value = None

  def fit(self, X, y):
    self.X = pd.DataFrame(X)
    self.y = np.array(y)
    # print('start fit')
    if (self.y.shape[0] <= self.min_leaf) or (np.unique(self.y).shape[0]==0):
      pass
    else:
      self.learn_the_feature()
      index_left = self.X.iloc[:,self.feature]  < self.feature_value
      index_right = self.X.iloc[:,self.feature]  >= self.feature_value
      left_tree  = self.X[index_left].loc[:,self.X.columns.values !=self.feature]
      right_tree = self.X[index_right].loc[:,self.X.columns.values !=self.feature]
      self.right = DecisionTree(self.min_leaf)
      self.right.fit(right_tree, self.y[index_right])
      # print('RIGHT')
      self.left = DecisionTree(self.min_leaf)
      self.left.fit(left_tree, self.y[index_left])
      # print('LEFT')


  def learn_the_feature(self):
    # print('start learn_the_feature')
    splits = []
    for ind in range(self.X.shape[1]):
      if np.var(self.X.iloc[:,ind]) == 0:
        # for those features that all the same for all data set some big number not to choose
        splits.append([1000,1000])
      else:
        splits.append(self.learn_best_split(self.X.iloc[:,ind].values))
    if len(splits) != 0:
      splits = np.array(splits)
      # print(splits)
      ind_min = splits[:,1].argmin()
      # print(ind_min, len(splits))
      # print(splits)
      self.feature = ind_min
      self.feature_value = splits[ind_min][0]
    else:
      print(self.X.shape)

  
  def learn_best_split(self, x):
    # print('start learn_best_split')
    # x - is a vector from self.X
    # returns the best value to split
    values = []
    errors = []
    pd_xy = pd.DataFrame([x,self.y],index=['x','y']).T.sort_values('x')
    # print('sorted')
    for ind in range(1, pd_xy.shape[0]):
        value = pd_xy.iloc[ind]['x']
        if (value < pd_xy.iloc[ind-1]['x']) and (pd_xy.iloc[ind-1]['y'] != pd_xy.iloc[ind]['y']):
          print('if')
          left = pd_xy.loc[pd_xy.x < value]['y'].values
          right = pd_xy.loc[pd_xy.x >= value]['y'].values
          values.append(value)
          errors.append(self.split_error(left, right))
    values = np.array(values)
    errors = np.array(errors)
    if len(errors) > 0:
      result = [values[errors.argmin()], errors.min()]
    else:
      result = [pd_xy.iloc[ind]['x'], 100]

    # choose the value that gives us the min split error
    return result
    

  def split_error(self, left, right):
    print('start split_error')
    return np.var(left) + np.var(right)


  def predict(self, X):
    pd_X = pd.DataFrame(X)
    prediction = np.zeros(pd_X.shape[0], dtype='float64')
    if self.feature is None:
      # print(self.y.mean(), prediction)
      return np.full(pd_X.shape[0],self.y.mean(), dtype='float64')
      # return prediction.fill(self.y.mean())
    else:
      # prediction = []
      index_left = pd_X.iloc[:,self.feature]  < self.feature_value
      index_right = pd_X.iloc[:,self.feature]  >= self.feature_value
      left_tree  = pd_X[index_left].loc[:,pd_X.columns.values !=self.feature]
      right_tree = pd_X[index_right].loc[:,pd_X.columns.values !=self.feature]
      # print('left')
      # np.put(prediction, index_left, self.left.predict(left_tree))
      prediction[index_left] = self.left.predict(left_tree)
      # print('right')
      prediction[index_right] = self.right.predict(right_tree)
      return prediction



In [124]:
my_dt = DecisionTree(5)
my_dt.fit(X_train, y_train)
y_pred_train = my_dt.predict(X_train)
y_pred_test = my_dt.predict(X_test)
# matrix = confusion_matrix(y_test, y_pred)
# plt.imshow(matrix)
# print(f'The total accuracy: {np.trace(matrix)/ matrix.sum():.4f}')
print(f'Var of the error on TRAIN set: {np.var(y_pred_train - y_train) :.4}')
print(f'Var of the error on TEST set: {np.var(y_pred_test - y_test) :.4}')


In [123]:
(y_pred-y_test)

array([ -1.2125    ,  -1.66666667,  -2.5       ,   6.67727273,
         4.77727273, -11.9       ,   2.08571429,  -6.3125    ,
         6.18      ,   9.        ,  -2.91428571,   4.58571429,
        -1.7       ,   4.18571429,   3.37727273, -10.22272727,
         1.75      ,   0.95      ,  -0.52      ,   2.78571429,
       -10.7       ,   2.37727273,   0.68571429,   0.67727273,
         0.28      ,  -2.61428571, -16.        ,   4.57727273,
         2.28      ,  -1.1       ,   2.28      ,   9.37727273,
        -0.62272727])

In [117]:
y_test

array([22.9, 23.4, 24.2, 19.3, 21.2, 30.8, 15.2, 28. , 15. , 24. , 20.2,
       12.7, 23.4, 13.1, 22.6, 36.2, 20. , 20.8, 21.7, 14.5, 35.4, 23.6,
       16.6, 25.3, 20.9, 19.9, 34.9, 21.4, 18.9, 22.8, 18.9, 16.6, 26.6])

In [87]:
ex = np.array([[1,2,3],[1,2,3],[1,4,5],[1,2,7],[1,2,8]])
ey = np.array([1,1,0,0,1])

k = np.empty(ex.shape[0], dtype='float64')
# np.empty_like(pd_X.shape[0], dtype='float64')
k.fill(ey.mean())
k, ex.shape[0]

(array([0.6, 0.6, 0.6, 0.6, 0.6]), 5)

In [5]:
from sklearn.datasets import load_boston
boston = load_boston()


In [39]:
X_train, X_test, y_train, y_test = train_test_split(boston.data[:100], boston.target[:100], test_size=0.33, random_state=42)

In [9]:
y_pred

22.386567164179105

In [3]:
from sklearn.datasets import load_boston
boston = load_boston()
X_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target, test_size=0.33, random_state=42)
to_test = np.array([1,5,10,100])
MSE = np.zeros(to_test.shape[0])
for i, min_leaf in enumerate(to_test):
  DT = DecisionTree(min_leaf)
  DT.fit(X_train, y_train)
  prediction = DT.predict(X_test)
  MSE[i] = np.mean((prediction - y_test)**2)
print(np.round(MSE))

/Users/tansla/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California hous

start fit
start learn_the_feature


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [290]:
X_train.pixel1.mean()

0.0

In [287]:
ex = np.array([[1,2,3],[1,2,3],[1,4,5],[1,2,7],[1,2,8]])
ey = np.array([1,1,0,0,1])
ex, ey

(array([[1, 2, 3],
        [1, 2, 3],
        [1, 4, 5],
        [1, 2, 7],
        [1, 2, 8]]),
 array([1, 1, 0, 0, 1]))

In [221]:
index_left = self.X.iloc[:,self.feature]  < self.feature_value
left_tree  = self.X[index_left].loc[:,self.X.columns.values !=self.feature]
self.left = DecisionTree(left_tree, self.y[index_left], self.min_leaf)

(5, 2)

In [223]:
ex2 = np.array([[1,2,3],[1,2,3],[1,4,5],[1,2,7],[1,2,8]])
ex2

array([[1, 2, 3],
       [1, 2, 3],
       [1, 4, 5],
       [1, 2, 7],
       [1, 2, 8]])

In [231]:
ex2[ex2[:,2].argmax()][0]

1

In [214]:
ex2.iloc[:,2].values

array([3, 3, 5, 7, 8])

In [206]:
ex[:,2]
pd_x = pd.DataFrame([ex[:,2],ey],index=['x','y']).T.sort_values('x')
pd_x

,x,y
0,3,1
1,3,1
2,5,0
3,7,0
4,8,1


In [ ]:
pd_x.loc[pd_x.x <= 2]['y'].values

In [ ]:
for value in pd_x['x'].unique():
    left = pd_x.loc[pd_x.x < value]['y'].values
    right = pd_x.loc[pd_x.x >= value]['y'].values
    print(value, left, right)

In [215]:
values = []
splits = []
pd_xy = pd.DataFrame([ex2.iloc[:,2].values,ey],index=['x','y']).T.sort_values('x')
for ind in range(1, pd_xy.shape[0]):
    value = pd_xy.iloc[ind]['x']
    if (value > pd_xy.iloc[ind-1]['x']) and (pd_xy.iloc[ind-1]['y'] != pd_xy.iloc[ind]['y']):
        left = pd_xy.loc[pd_xy.x < value]['y'].values
        right = pd_xy.loc[pd_xy.x >= value]['y'].values

        values.append(value)
        splits.append( left.mean() + right.mean()-1)


In [216]:
values, splits

([5, 8], [0.33333333333333326, 0.5])

In [205]:
values[np.array(splits).argmax()]

8

## 3. Using Decision Tree for Digits Classification
Remeber the MNIST dataset used - you will now test the power of decision trees on this problem.
This time you are given a free hand in choosing the test and train set sizes, model parameters (such as gain function and constraints over the trees) and features (whether to use binary pixel values or the original continous gray value).
  - You can use `sklearn.tree.DecisionTreeClassifier`
- Once you are satisfied with the model parameters, plot the importance of each of the pixels to the final decision.
- Last, estimate the class assignment probabilities for all the correctly classified and misclassified examples in your test data.
- Discuss your results.

In [ ]:
from sklearn.datasets import fetch_openml
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)

In [ ]:
X.shape

In [235]:
# code and answer go here
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=317,stratify=y)


Baseline from NB exercise: The total accuracy: 0.8370

In [ ]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_test)
matrix = confusion_matrix(y_test, y_pred)
plt.imshow(matrix)
print(f'The total accuracy: {np.trace(matrix)/ matrix.sum():.4f}')
print(f'DecisionTreeClassifier Score on TRAIN set: {dtc.score(X_train, y_train):.4}')
print(f'DecisionTreeClassifier Score on TEST set: {dtc.score(X_test, y_test):.4}')
fi = dtc.feature_importances_

In [ ]:

plt.imshow(fi.reshape(28,28))

In [ ]:
plt.imshow(np.where(fi>0.005,1,0).reshape(28,28))

In [ ]:
dtc.predict_proba(X_test).sum(axis=1).max()

In [ ]:
# Cheking for different values of feature importance how many features are there
a = 0.005
plt.plot(fi[fi>a])
print(fi[fi>a].shape)

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'max_features':(3,8,11, 20,32), 'max_depth': range(5,50)}

model = DecisionTreeClassifier()
grid_GBC = GridSearchCV(model, parameters)
grid_GBC.fit(X_train, y_train)

best_params = grid_GBC.best_params_

print(" Results from Grid Search " )
print(f'The best estimator across ALL searched params on TRAIN set: {grid_GBC.best_estimator_}')
print(f'The best score across ALL searched params on TRAIN set: {grid_GBC.best_score_:.4f}')
print(f'The best parameters across ALL searched params on TRAIN set: {best_params}')

In [ ]:
dtc = DecisionTreeClassifier(max_features=32, max_depth=16)
dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_test)
matrix = confusion_matrix(y_test, y_pred)
plt.imshow(matrix)
print(f'The total accuracy: {np.trace(matrix)/ matrix.sum():.4f}')
print(f'DecisionTreeClassifier Score on TRAIN set: {dtc.score(X_train, y_train):.4}')
print(f'DecisionTreeClassifier Score on TEST set: {dtc.score(X_test, y_test):.4}')
fi = dtc.feature_importances_

In [ ]:
np.unique(dtc.predict_proba(X_test))

In [ ]:
plt.imshow(fi.reshape(28,28))

In [ ]:
# Let's binarize the data and train a new tree:

model_param = 150
X_train_bool = np.where(X_train>model_param,1,0)
X_test_bool = np.where(X_test>model_param,1,0)

In [ ]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train_bool, y_train)
y_pred = dtc.predict(X_test_bool)
matrix = confusion_matrix(y_test, y_pred)
plt.imshow(matrix)
print(f'The total accuracy: {np.trace(matrix)/ matrix.sum():.4f}')
print(f'DecisionTreeClassifier Score on TRAIN set: {dtc.score(X_train_bool, y_train):.4}')
print(f'DecisionTreeClassifier Score on TEST set: {dtc.score(X_test_bool, y_test):.4}')
fi = dtc.feature_importances_

In [ ]:
# Cheking for different values of feature importance how many features are there
a = 0.02
plt.plot(fi[fi>a])
print(fi[fi>a].shape)

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'max_features':range(5,35,3), 'max_depth': range(5,50,3)}

model = DecisionTreeClassifier()
grid_GBC = GridSearchCV(model, parameters)
grid_GBC.fit(X_train_bool, y_train)

best_params = grid_GBC.best_params_

print(" Results from Grid Search " )
print(f'The best estimator across ALL searched params on TRAIN set: {grid_GBC.best_estimator_}')
print(f'The best score across ALL searched params on TRAIN set: {grid_GBC.best_score_:.4f}')
print(f'The best parameters across ALL searched params on TRAIN set: {best_params}')

In [ ]:
dtc = DecisionTreeClassifier(max_features=32, max_depth=38)
dtc.fit(X_train_bool, y_train)
y_pred = dtc.predict(X_test_bool)
matrix = confusion_matrix(y_test, y_pred)
plt.imshow(matrix)
print(f'The total accuracy: {np.trace(matrix)/ matrix.sum():.4f}')
print(f'DecisionTreeClassifier Score on TRAIN set: {dtc.score(X_train_bool, y_train):.4}')
print(f'DecisionTreeClassifier Score on TEST set: {dtc.score(X_test_bool, y_test):.4}')